In [4]:
!pip install opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 1.2 MB/s eta 0:00:00m eta 0:00:010:00:02m


In [6]:
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 1.2 MB/s eta 0:00:00m eta 0:00:010:00:11
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 2.6 MB/s eta 0:00:003.4 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 2.5 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 2.8 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 3.2 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 3.2 MB/s eta 0:00:003.9 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 3.7 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 4.6 MB

In [7]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model

2023-12-20 21:17:54.120107: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-20 21:17:54.120244: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-20 21:17:54.127493: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-20 21:17:54.161898: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-20 21:17:57.867148: W tensorflow/compiler/tf2

In [8]:
def load_and_preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))  # Resize image to fit VGG19 input size
    image = preprocess_input(image)
    return image

In [9]:
def random_sample_pixels(image, sample_size=625):
    height, width, _ = image.shape
    indices = np.random.choice(height * width, size=sample_size, replace=False)
    sampled_pixels = np.array([image[idx // width, idx % width] for idx in indices])
    return sampled_pixels

In [20]:
def count_pixels_per_color_region(sampled_pixels, colorspace_divisions=6): #8):
    color_counts = np.zeros((colorspace_divisions, colorspace_divisions, colorspace_divisions))

    for pixel in sampled_pixels:
        r, g, b = pixel.astype(int)
        color_counts[r // (256 // colorspace_divisions),
                     g // (256 // colorspace_divisions),
                     b // (256 // colorspace_divisions)] += 1

    return color_counts.flatten()

# Sample image

In [11]:
image_path = 'examples/inputs/brad_pitt.jpg'

# Step 1: Load image

In [12]:
image = load_and_preprocess_image(image_path)

# Step 2: Random sample 625 pixels

In [13]:
sampled_pixels = random_sample_pixels(image)

# Step 3: Count pixel counts per color region

In [21]:
color_counts = count_pixels_per_color_region(sampled_pixels)

# Results

In [22]:
print("Sampled Pixels:\n", sampled_pixels)
print("\nColor Counts per Color Region:\n", color_counts)

Sampled Pixels:
 [[ 121.061      115.221      105.32     ]
 [ 123.061      112.221      105.32     ]
 [ 126.061      118.221      114.32     ]
 ...
 [  33.060997    50.221      108.32     ]
 [ -26.939003    -2.7789993   55.32     ]
 [ -88.939     -102.779     -106.68     ]]

Color Counts per Color Region:
 [  3.   6.  27.   0.   0.   0.   0.   1.  28.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   4.   0.   0.   0.   0.  21.   0.   0.   0.   0.   0.   0.
   1.   1.  26.   5.   0.   0.   0.   0.   3.   9.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   2. 201.   3.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  41.   0.   0.   0.
   0.   0.   0.  35.  10.   1.   0.   0.   0.   0.   

In [23]:
len(color_counts)

216